This program assigns roles to team members for the project. 

Conditions:
1. Team will rotate team roles in "cycles".
2. The length of each cycle may be two weeks or more, as determined by the team.
3. Each role has an expert who will guide the team members assigned to the role. 
4. Each team member may be assigned to more than one role at any given time. 
5. No team members, besides the experts, will be assigned to the same role in two consecutive cycles. 
6. Experts may be assigned to other roles just like any other team member. 

In [114]:
import pandas as pd
import numpy as np
import random
random.seed(42)

Instantiate list of members

In [115]:
members = [
            "Oluwaseyi Adaramola",
            "Farook Ajose",
            "Malik Akintola",
            "Ibukun Akinleye",
            "Samson Ilesanmi",
            "Oluwaseunfunmi Alarima",
            "Damilola Omokehinde",
            "Samuel Olayiwola",
            "Adejuwon Adegite",
            "Kafilat Akinyemi",
            "Chibike Enibe",
            "Ayodeji Abatan",
            "Divine Nkwocha",
            "AbdulQahhar Okenla",
        ]

members

['Oluwaseyi Adaramola',
 'Farook Ajose',
 'Malik Akintola',
 'Ibukun Akinleye',
 'Samson Ilesanmi',
 'Oluwaseunfunmi Alarima',
 'Damilola Omokehinde',
 'Samuel Olayiwola',
 'Adejuwon Adegite',
 'Kafilat Akinyemi',
 'Chibike Enibe',
 'Ayodeji Abatan',
 'Divine Nkwocha',
 'AbdulQahhar Okenla']

Instantiate available roles

In [116]:
roles = pd.DataFrame({
    "role": ["Data Analyst (Python)",
            "Data Scientist/Statistician",
            "Visualization Specialist (Power BI)",
            "Technical Writer",
            "Publicity Coordinator",],
    "num_people": [4, 4, 4, 4, 3],
    "expert": ["Malik Akintola",
               "Ayodeji Abatan",
               "Samson Ilesanmi",
               "Farook Ajose",
               "AbdulQahhar Okenla"]
    })
roles

,role,num_people,expert
0,Data Analyst (Python),4,Malik Akintola
1,Data Scientist/Statistician,4,Ayodeji Abatan
2,Visualization Specialist (Power BI),4,Samson Ilesanmi
3,Technical Writer,4,Farook Ajose
4,Publicity Coordinator,3,AbdulQahhar Okenla


Write function to assign roles

In [117]:
num_cycles = 6

def assign_roles(row, cycles=num_cycles, team=members):
    """Assign team members to different roles for each rotation cycle, ensuring that an expert in the area is present to guide them. 

    Args:
        row (pandas.Series): A row of the `roles` dataframe containing the role details.
        cycles (int, optional): Number of rotation cycles to assign the roles for. Defaults to the value of `num_cycles`.
        team (list, optional): A list of team members to select from. Defaults to the value of `members`.

    Returns:
        expert_cycle_dict (dict): A dictionary containing the assigned members for each cycle, with keys in the format "Cycle x" where x is the cycle number.
    """
    
    # Remove expert from sample list 
    expert = row['expert']
    no_expert_team = team.copy()
    no_expert_team.remove(expert)

    # Create dictionary of assigned members for each cycle
    cycle_dict = {
        f"Cycle {cycle}":  np.random.choice(no_expert_team, size=row["num_people"]-1, replace=False)
        for cycle in range(1, cycles+1)
    }
    
    # Remove common values between consecutive cycles
    for i in range(1, cycles):
        current_cycle = cycle_dict[f"Cycle {i}"]
        next_cycle = cycle_dict[f"Cycle {i+1}"]
        common_members = np.intersect1d(current_cycle, next_cycle)
        for member in common_members:
            new_member = np.random.choice(no_expert_team, size=1)[0]
            while new_member in next_cycle:
                new_member = np.random.choice(no_expert_team, size=1)[0]
            next_cycle = np.where(next_cycle == member, new_member, next_cycle)
        cycle_dict[f"Cycle {i+1}"] = next_cycle
        
    # Append expert to list and convert to string    
    expert_cycle_dict = {
        f"Cycle {cycle}":  ", ".join(np.append(expert, cycle_dict[f"Cycle {cycle}"]))
        for cycle in range(1, cycles+1)
    }

    return expert_cycle_dict
        
# Test function to see outputs.     
assign_roles(roles.iloc[2, :])

{'Cycle 1': 'Samson Ilesanmi, Oluwaseunfunmi Alarima, Chibike Enibe, Oluwaseyi Adaramola',
 'Cycle 2': 'Samson Ilesanmi, Divine Nkwocha, Kafilat Akinyemi, Oluwaseunfunmi Alarima',
 'Cycle 3': 'Samson Ilesanmi, Chibike Enibe, Adejuwon Adegite, Farook Ajose',
 'Cycle 4': 'Samson Ilesanmi, AbdulQahhar Okenla, Oluwaseunfunmi Alarima, Malik Akintola',
 'Cycle 5': 'Samson Ilesanmi, Adejuwon Adegite, Oluwaseyi Adaramola, Farook Ajose',
 'Cycle 6': 'Samson Ilesanmi, AbdulQahhar Okenla, Damilola Omokehinde, Ibukun Akinleye'}

Update dataframe with function

In [118]:

roles[[f"Cycle {cycle}" for cycle in range(1, num_cycles+1)]] = roles.apply(func=assign_roles, axis=1, result_type='expand')
roles

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Kafilat Akinyemi, Adejuwon Ade...","Malik Akintola, Samson Ilesanmi, Farook Ajose,...","Malik Akintola, Divine Nkwocha, Chibike Enibe,...","Malik Akintola, Samuel Olayiwola, Adejuwon Ade...","Malik Akintola, Oluwaseyi Adaramola, Ayodeji A...","Malik Akintola, Chibike Enibe, Samson Ilesanmi..."
1,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, Samuel Olayiwola, AbdulQahhar ...","Ayodeji Abatan, Samson Ilesanmi, Farook Ajose,...","Ayodeji Abatan, Damilola Omokehinde, AbdulQahh...","Ayodeji Abatan, Samuel Olayiwola, Oluwaseunfun...","Ayodeji Abatan, Malik Akintola, AbdulQahhar Ok...","Ayodeji Abatan, Samson Ilesanmi, Malik Akintol..."
2,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, AbdulQahhar Okenla, Malik Aki...","Samson Ilesanmi, Oluwaseunfunmi Alarima, Samue...","Samson Ilesanmi, Samuel Olayiwola, Chibike Eni...","Samson Ilesanmi, Kafilat Akinyemi, Oluwaseyi A...","Samson Ilesanmi, Ayodeji Abatan, Ibukun Akinle...","Samson Ilesanmi, Damilola Omokehinde, Divine N..."
3,Technical Writer,4,Farook Ajose,"Farook Ajose, Divine Nkwocha, Oluwaseunfunmi A...","Farook Ajose, Malik Akintola, Damilola Omokehi...","Farook Ajose, Oluwaseyi Adaramola, Divine Nkwo...","Farook Ajose, Samson Ilesanmi, Ibukun Akinleye...","Farook Ajose, Divine Nkwocha, Malik Akintola, ...","Farook Ajose, Ayodeji Abatan, Samson Ilesanmi,..."
4,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Malik Akintola, Oluwaseunf...","AbdulQahhar Okenla, Kafilat Akinyemi, Ayodeji ...","AbdulQahhar Okenla, Adejuwon Adegite, Malik Ak...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Ayode...","AbdulQahhar Okenla, Damilola Omokehinde, Faroo...","AbdulQahhar Okenla, Divine Nkwocha, Adejuwon A..."


Save dataframe to CSV

In [119]:
# roles.to_csv("assigned_roles.csv", index=False)